Primer paso es leer BD del servidor donde se tenga la tabla de los pesometros antiguos con problemas.
En este caso es el servidor de google

In [14]:
import mysql.connector
import time
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter


db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")
consulta="SHOW TABLES"
cur.execute(consulta)
for row in cur.fetchall():
    print (str(row[0]))
print (" ")


conexion a gc
camiones_reg
cargador_reg
pesometroreparado
pesometros
 


Los datos en bruto estan en la tabla pesometro columna acc1 pesometro integral y acc2 pesometro grava 3/4.
row[1]=fecha, row[5]=integral, row[6]=grava3/4

In [15]:
consulta="DESCRIBE pesometros"
cur.execute(consulta)
for row in cur.fetchall():
    print (str(row[0]))
print (" ")


id
fecha
tph1
tph2
tph3
acc1
acc2
acc3
diff1
diff2
diff3
 


vamos a leer el primer y ultimo registro de la tabla pesometro, en la consulta anterior se muestran los nombre de las columnas


In [25]:
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")

integralx=[]
gravax=[]
fechaIni="'2019-08-07 08:00:03'"
fechaFin="'2019-08-16 22:47:00'" 
#consula
tabla="pesometros"
consulta="SELECT * FROM "+tabla+" WHERE fecha >= "+fechaIni+ "ORDER BY id ASC LIMIT 1"
print(consulta)
cur.execute(consulta)
print ("El primer registro de la consulta ")
for row in cur.fetchall():
     print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
     integralx.append(row[5])
     gravax.append(row[6])
print (" ")
print ("El ultimo registro de la consulta ")
tabla="pesometros"
consulta="SELECT * FROM "+tabla+" WHERE fecha <= "+fechaFin+ "ORDER BY id DESC LIMIT 1"
cur.execute(consulta)
for row in cur.fetchall():
     print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
     integralx.append(row[5])
     gravax.append(row[6])
print (" ")


conexion a gc
SELECT * FROM pesometros WHERE fecha >= '2019-08-07 08:00:03'ORDER BY id ASC LIMIT 1
El primer registro de la consulta 
54159986 2019-08-07 19:20:13 -1 -1
 
El ultimo registro de la consulta 
54314731 2019-08-16 22:45:07 0 0
 


Ahora vamos a reparar los datos


In [27]:
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")
fechaIni="'2019-08-07 08:00:03'"
fechaFin="'2019-08-16 22:47:00'" 
#consula
tabla="pesometros"

integral=[]
grava=[]
restaint=integralx[0]
restagrav=gravax[0]
consulta="select * FROM "+tabla+" WHERE fecha between "+fechaIni+" and "+fechaFin+" "  
print(consulta)
cur.execute(consulta)

anteriorint=0  # registro anterior de la base antigua
anteriorgrav=0
contador=0
print(integralx)
print(gravax)



conexion a gc
select * FROM pesometros WHERE fecha between '2019-08-07 08:00:03' and '2019-08-16 22:47:00' 
[254724709600, 265377798100]
[84099887700, 87408469800]


In [ ]:
print("vamos a comenzar la revision e insercion")
consulta="select * FROM "+tabla+" WHERE fecha between "+fechaIni+" and "+fechaFin+" "  
print(consulta)
cur.execute(consulta)

anteriorint=0  # registro anterior de la base antigua
anteriorgrav=0
contador=0
print(integralx)
print(gravax)


for row in cur.fetchall():
     #print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
    if contador==0:
         anteriorint=row[5]
         anteriorgrav=row[6]
         anteriorintsql=0
         anteriorgravasql=0
    if row[5]-anteriorint>30:

        print("entro aca")
        restaint=restaint+row[5]-anteriorint
        print ("cambio de Integral")
    if row[6]-anteriorgrav>30:
        restagrav=restagrav+row[6]-anteriorgrav
              
   
    print(str(contador)+"  "+str(row[1])+"   "+str(row[5])+"   "+str(row[5]-restaint)+"---------"+str(row[6])+"   "+str(row[6]-restagrav))
    integral.append(row[2]-restaint)
    grava.append(row[3]-restagrav)
    #aca debo hacer el insert a la nueva tabla
    consulta="INSERT INTO `pesometroreparado` (`hora`, `acc1`, `acc2`, `acc3`)VALUES ('"+str(row[1])+"', '"+str(row[5]-restaint-anteriorintsql)+"', '"+str(row[6]-restagrav-anteriorgravasql)+"', NULL);"
    #print(consulta)
    #print(anteriorint)
    cur.execute(consulta)
    db.commit()
    anteriorint=row[5]
    anteriorgrav=row[6]
    anteriorintsql=row[5]-restaint
    anteriorgravasql=row[6]-restagrav
    contador=contador+1

print("la produccion de integral es : ")
print(str(integral[len(integral)-1]))
print("la produccion de integral es en toneladas: ")
print(str(integral[len(integral)-1]*100/1000))

print("la produccion de grava es : ")
print(str(grava[len(grava)-1]))

print("la produccion de grava en toneladas es : ")
print(str(grava[len(grava)-1]*100/1000))